In [1]:
## Code for running Wizardry-esque early dungeon crawl games
## by Nicholas Smith
## Copyright (c) 2025

In [2]:
## import statements go here
import nbformat
import importlib
import import_ipynb
import io
import os
import sys
import pygame

pygame 2.6.1 (SDL 2.28.4, Python 3.12.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
## Global constants

WALL    = 0b0
ROOM    = 0b1
VISITED = 0b10
ENEMY   = 0b100
ROOMKEY = 0b1000

## Global variables

currentmap = None                                       # Current level map
mapWidth = 0                                            # Current map's width
mapHeight = 0                                           # Current map's height
playerLocation = (8, 4)                                   # Current coordinates of player
playerFacing = 1                                        # Direction player is facing (1 for North, 2 for East, 3 for South, 4 for West)

savefile = None                                         # Current loaded save file

enemies = []                                            # List of all enemy Sprites
items = []                                              # List of all item Sprite
VALID_FAMILIES = ['player','enemy','item']              # list of all Sprite families in game
VALID_TYPES = ['player','enemy','item']                 # List of all Sprite types

screen = None                                           # pygame display surface
pointsone = None                                        # Points denoting far corners of player's space on the map
pointstwo = None                                        # Points denoting far corners of space one in front of player on the map
pointsthree = None                                      # Points denoting far corners of space two in front of player on the map
pointsfour = None                                       # Points denoting far corners of space three in front of player on the map
screenWidth = 0                                         # Width of pygame window in pixels
screenHeight = 0                                        # Height of pygame window in pixels

In [4]:
## Main launch method

def main():
    global currentmap
    global playerfacing

    # Initialize the pygame window
    initialize()
    
    # Load up the first map
    currentmap = loadMap("MovementTestLevel.lvlmap")

    # Ensure player is pointing North
    playerfacing = 1

    # Draw a first-person view of the initial map state
    drawView(getVisible())

    # Begin the event-capture game loop
    gameLoop()
    # print("All good")
    return

In [5]:
## Load current map into memory from disk
## Returns a Map object

def loadMap(mapfile):

    global mapWidth
    global mapHeight

    mapstream = open(mapfile, "r", encoding="utf-8")
    try:
        # Ensure the filetype is correct
        teststring = mapstream.readline()
        if teststring != "Wizdrive map\n":
            raise ValueError("Loaded file was not a valid map.")

        # Split map into a two-dimensional array
        map = mapstream.read()
        map = map.split("\n")
        map[len(map)-1].strip()
        for l, line in enumerate(map):
            map[l] = line.split(",")
    
    # Set mapHeight and mapWidth based on this new array
        mapHeight = len(map)
        mapWidth = len(map[0])
    # Sanity check; all rows must be the length of mapWidth and there must be mapHeight rows
        for l in range(mapHeight):
            for c in range(mapWidth):
                testvar = map[l][c]
                map[l][c] = int(map[l][c])
    except Exception:
        sys.exit(1)
    return map

In [6]:
## Main game loop

def gameLoop():
    run = True
    while run:
        # Poll continuously for keyboard input
        # When one is found, parse it and perform the action
        # for gameEvent in pygame.event.get():
        for gameEvent in pygame.event.get():
            
            # If a key was pressed, toss it to the keyboardHandler() method
            if gameEvent.type == pygame.KEYDOWN:
                keyboardHandler(gameEvent.key)
            
            # If a quit command was sent, end the loop and exit the program
            elif gameEvent.type == pygame.QUIT:
                run = False
                pygame.quit()
    sys.exit(0)
    # Enemies take turns
    # Repeat
    return

In [ ]:
## Code to get a map of visible spaces from the current map and facing. Values are a 4x3 matrix of the spaces immediately
## in front of the player, with the "front" determined by the value of global varible playerfacing.

def getVisible():
    visible = [[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]]
    # start with playerFacing 1 (North)
    match playerFacing:
        case 1:
            for r in range(playerLocation[0]-3, playerLocation[0]+1):
                y = r - (playerLocation[0]-3)
                for c in range(playerLocation[1]-1, playerLocation[1]+2):
                    x = c - (playerLocation[1]-1)
                    try:
                        visible[y][x] = currentmap[r][c]
                    except IndexError:
                        visible[y][x] = 0
        # Untested
        case 2:
            for r in range(playerLocation[0]-1, playerLocation[0]+2):
                x = r - (playerLocation[0]-1)
                for c in range(playerLocation[1], playerLocation[1]+4):
                    y = c - playerLocation[1]
                    try:
                        visible[y][2-x] = currentmap[c][r]
                    except IndexError:
                        visible[y][2-x] = 0
        # Untested
        case 3:
            for r in range(playerLocation[0], playerLocation[o]-4, step=-1):
                y = r - (playerLocation[0])
                for c in range(playerLocation[1]+1, playerLocation[1]-2, step=-1):
                    x = c - (playerLocation[1]-1)
                    try:
                        visible[3-y][2-x] = currentmap[c][r]
                    except IndexError:
                        visible[3-y][2-x] = 0
        # Untested
        case 4:
            for r in range(playerLocation[0]+1, playerLocation[0]-2, step=-1):
                y = r - playerLocation[0]
                for c in range(playerLocation[1]-3, playerlocation[1] + 1):
                    x = 1
                try:
                    visible[y][x] = currentmap[r][c]
                except IndexError:
                    visible[y][x] = 0
        case _:
            raise ValueError
    print(visible[0])
    print(visible[1])
    print(visible[2])
    print(visible[3])

    return visible

In [8]:
##################################### USER CONTROL CODE #######################################



In [9]:
## Handle keyboard input

def keyboardHandler(pressed):
    match pressed:
        case pygame.K_UP:
            print("K_UP pressed")
        case pygame.K_DOWN:
            print("K_DOWN pressed")
            drawView(getVisible())
        case pygame.K_LEFT:
            print("K_LEFT pressed")
        case pygame.K_RIGHT:
            print("K_RIGHT pressed")
        case _:
            print("Not an arrow key")
    return

In [10]:
###################################### GRAPHICS CODE #######################################



In [11]:
## Initialize pygame and set graphics constants based on size of window

def initialize():
    # Method uses all global graphics variables
    global screen
    global pointsone
    global pointstwo
    global pointsthree
    global pointsfour
    global screenWidth
    global screenHeight

    # initialize Pygame
    pygame.init()
    (screenWidth, screenHeight) = (800, 800)
    pointsone = ((screenWidth*.20, screenHeight*.20), (screenWidth*.80, screenHeight*.20), (screenWidth*.2, screenHeight*.8),(screenWidth*.8, screenHeight*.8))
    pointstwo = ((screenWidth*.4, screenHeight*.40), (screenWidth*.60, screenHeight*.40), (screenWidth*.4, screenHeight*.6),(screenWidth*.6, screenHeight*.6))
    pointsthree = ((screenWidth*.45, screenHeight*.45), (screenWidth*.55, screenHeight*.45), (screenWidth*.45, screenHeight*.55),(screenWidth*.55, screenHeight*.55))
    pointsfour = ((screenWidth*.455, screenHeight*.455), (screenWidth*.525, screenHeight*.455), (screenWidth*.455, screenHeight*.525),(screenWidth*.525, screenHeight*.525))

    screen = pygame.display.set_mode(size=(screenWidth, screenHeight))
    screen.fill("purple")
    pygame.display.flip()

In [12]:
## Draw cell for currently occupied space

def __drawZero():
    pygame.draw.line(screen, "black", (0, 0), pointsone[0])
    pygame.draw.line(screen, "black", (0, screenHeight), pointsone[2])
    pygame.draw.line(screen, "black", (screenWidth, 0), pointsone[1])
    pygame.draw.line(screen, "black", (screenWidth, screenHeight), pointsone[3])
    pygame.draw.line(screen, "black", pointsone[0], pointsone[1])
    pygame.draw.line(screen, "black", pointsone[0], pointsone[2])
    pygame.draw.line(screen, "black", pointsone[1], pointsone[3])
    pygame.draw.line(screen, "black", pointsone[2], pointsone[3])
    return

## Draw cell to left of player
def __drawZeroLeft():
    pygame.draw.line(screen, "black", pointsone[0], (0, pointsone[0][1]))
    pygame.draw.line(screen, "black", pointsone[2], (0, pointsone[2][1]))
    return

## Draw cell to right of player
def __drawZeroRight():
    pygame.draw.line(screen, "black", pointsone[1], (screenWidth, pointsone[1][1]))
    pygame.draw.line(screen, "black", pointsone[3], (screenWidth, pointsone[3][1]))
    return

## Draw cell one space ahead of player
def __drawOne():
    pygame.draw.line(screen, "black", pointsone[0], pointstwo[0])
    pygame.draw.line(screen, "black", pointsone[2], pointstwo[2])
    pygame.draw.line(screen, "black", pointsone[1], pointstwo[1])
    pygame.draw.line(screen, "black", pointsone[3], pointstwo[3])
    pygame.draw.line(screen, "black", pointstwo[0], pointstwo[1])
    pygame.draw.line(screen, "black", pointstwo[0], pointstwo[2])
    pygame.draw.line(screen, "black", pointstwo[1], pointstwo[3])
    pygame.draw.line(screen, "black", pointstwo[2], pointstwo[3])
    return

## Draw cell one left
def __drawOneLeft():
    pygame.draw.line(screen, "black", pointstwo[0], (pointsone[0][0], pointstwo[0][1]))
    pygame.draw.line(screen, "black", pointstwo[2], (pointsone[2][0], pointstwo[2][1]))
    return

## Draw cell one right
def __drawOneRight():
    pygame.draw.line(screen, "black", pointstwo[1], (pointsone[1][0], pointstwo[1][1]))
    pygame.draw.line(screen, "black", pointstwo[3], (pointsone[3][0], pointstwo[3][1]))
    return

## Draw cell two spaces ahead of player
def __drawTwo():
    pygame.draw.line(screen, "black", pointstwo[0], pointsthree[0])
    pygame.draw.line(screen, "black", pointstwo[2], pointsthree[2])
    pygame.draw.line(screen, "black", pointstwo[1], pointsthree[1])
    pygame.draw.line(screen, "black", pointstwo[3], pointsthree[3])
    pygame.draw.line(screen, "black", pointsthree[0], pointsthree[1])
    pygame.draw.line(screen, "black", pointsthree[0], pointsthree[2])
    pygame.draw.line(screen, "black", pointsthree[1], pointsthree[3])
    pygame.draw.line(screen, "black", pointsthree[2], pointsthree[3])
    return

## Draw cell two left
def __drawTwoLeft():
    pygame.draw.line(screen, "black", pointsthree[0], (pointstwo[0][0], pointsthree[0][1]))
    pygame.draw.line(screen, "black", pointsthree[2], (pointstwo[2][0], pointsthree[2][1]))
    return

## Draw cell two right
def __drawTwoRight():
    pygame.draw.line(screen, "black", pointsthree[1], (pointstwo[1][0], pointsthree[1][1]))
    pygame.draw.line(screen, "black", pointsthree[3], (pointstwo[3][0], pointsthree[3][1]))
    return

In [13]:
## Draw the first-person view of the current Player location and directional orientation
## based on a submitted 4x3 local map

def drawView(localmap):
    __drawZero()
    if localmap[3][0]:
        __drawZeroLeft()
    if localmap[3][2]:
        __drawZeroRight()
    if localmap[2][1]:
        __drawOne()
        if localmap[2][0]:
            __drawOneLeft()
        if localmap[2][2]:
            __drawOneRight()
        if localmap[1][1]:
            __drawTwo()
            if localmap[1][0]:
                __drawTwoLeft()
            if localmap[1][2]:
                __drawTwoRight()
    pygame.display.flip()
    
    

In [14]:
## Program launch code
if __name__ == '__main__':
    print("Pygame works")
    main()

Pygame works
[0, 1, 0]
[1, 1, 1]
[0, 1, 0]
[1, 7, 0]


SystemExit: 0

c:\Users\nmsmi\OneDrive\Documents\Coding projects\WizardryEngine\env\Lib\site-packages\IPython\core\interactiveshell.py:3707: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
